# Recurring Neural Networks com Keras

## Análise de sentimento a partir de resenhas de filmes

Na verdade, é um ótimo exemplo do uso de RNN. O conjunto de dados que estamos usando consiste em resenhas de filmes geradas por usuários e classificação de se o usuário gostou ou não do filme com base em sua classificação associada.

Mais informações sobre o conjunto de dados estão aqui:

https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification

Então, vamos usar um RNN para fazer análise de sentimentos em resenhas de filmes em texto!

Pense em como isso é incrível. Vamos treinar uma rede neural artificial para "ler" resenhas de filmes e adivinhar se o autor gostou ou não do filme.

Uma vez que a compreensão da linguagem escrita requer o acompanhamento de todas as palavras de uma frase, precisamos de uma rede neural recorrente para manter uma "memória" das palavras que vieram antes, uma vez que "lê" as sentenças ao longo do tempo.

Em particular, usaremos células LSTM (Long Short-Term Memory) porque não queremos "esquecer" palavras muito rapidamente - as palavras no início de uma frase podem afetar significativamente o significado dessa frase.

Vamos começar importando as coisas que precisamos:

In [1]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb
import warnings
warnings.filterwarnings('ignore')

Agora importe nossos dados de treinamento e teste. Especificamos que nos importamos apenas com as 20.000 palavras mais populares no conjunto de dados para manter as coisas um pouco fáceis de gerenciar. O conjunto de dados inclui 5.000 revisões de treinamento e 25.000 revisões de testes por algum motivo.

In [2]:
print('Carregando os dados...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=20000)

Loading data...


Vamos sentir como são esses dados. Vamos ver o primeiro recurso de treinamento, que deve representar uma crítica escrita de filme:

In [3]:
x_train[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 19193,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 10311,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 12118,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,

Isso não parece uma resenha de filme! Mas este conjunto de dados poupou-lhe muitos problemas - eles já converteram palavras para índices baseados em números inteiros. As letras reais que compõem uma palavra realmente não importam no que diz respeito ao nosso modelo, o que importa são as próprias palavras - e nosso modelo precisa de números para trabalhar, não de letras.

Portanto, lembre-se de que cada número nos recursos de treinamento representa alguma palavra específica. É uma chatice que não podemos apenas ler as resenhas em inglês como um teste para ver se a análise de sentimentos está realmente funcionando.

Como são as etiquetas?

In [4]:
y_train[0]

1

Eles são apenas 0 ou 1, o que indica se o revisor disse que gostou do filme ou não.

Então, para recapitular, temos várias resenhas de filmes que foram convertidas em vetores de palavras representadas por inteiros e uma classificação de sentimento binário para aprendermos.

O RNN pode explodir rapidamente, então, novamente, para manter as coisas fáceis de gerenciar em nosso pequeno PC, vamos limitar as revisões às primeiras 80 palavras:

In [5]:
x_train = sequence.pad_sequences(x_train, maxlen=80)
x_test = sequence.pad_sequences(x_test, maxlen=80)

Agora vamos configurar nosso modelo de rede neural! Considerando o quão complicada é uma rede neural recorrente do LSTM, é realmente incrível como isso é fácil com o Keras.

Vamos começar com uma camada de embedding - isso é apenas uma etapa que converte os dados de entrada em vetores densos de tamanho fixo que é mais adequado para uma rede neural. Você geralmente vê isso em conjunto com dados de texto baseados em índice, como temos aqui. Os 20.000 indicam o tamanho do vocabulário (lembre-se que dissemos que só queríamos as 20.000 palavras mais importantes) e 128 é a dimensão de saída de 128 unidades.

Em seguida, basta configurar uma camada LSTM para o próprio RNN. É tão fácil. Nós especificamos 128 para combinar com o tamanho de saída da camada Embedding e os termos de dropout para evitar overfitting, aos quais os RNNs são particularmente propensos.

Finalmente, precisamos apenas resumi-lo a um único neurônio com uma função de ativação sigmoide para escolher nossa classificação de sentimento binay de 0 ou 1.

In [6]:
model = Sequential()
model.add(Embedding(20000, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

Como este é um problema de classificação binária, usaremos a função de perda binary_crossentropy. E o otimizador Adam geralmente é uma boa escolha (sinta-se à vontade para experimentar outras pessoas).

In [7]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Agora vamos realmente treinar nosso modelo. Os da RNN, como os da CNN, são muito pesados em recursos. Manter o tamanho do lote relativamente pequeno é a chave para que isso funcione no seu PC. Na verdade, você estaria aproveitando as GPUs instaladas em vários computadores em um cluster para tornar essa escala muito melhor.

## Aviso

Isso levará muito tempo para ser executado, mesmo em um PC rápido! Não execute o próximo bloco, a menos que você esteja preparado para dedicar seu computador por uma hora ou mais.

In [8]:
model.fit(x_train, y_train,
          batch_size=32,
          epochs=15,
          verbose=2,
          validation_data=(x_test, y_test))

C:\Users\Frank\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\tensorflow\python\ops\gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/15
 - 139s - loss: 0.6580 - acc: 0.5869 - val_loss: 0.5437 - val_acc: 0.7200
Epoch 2/15
 - 138s - loss: 0.4652 - acc: 0.7772 - val_loss: 0.4024 - val_acc: 0.8153
Epoch 3/15
 - 136s - loss: 0.3578 - acc: 0.8446 - val_loss: 0.4024 - val_acc: 0.8172
Epoch 4/15
 - 134s - loss: 0.2902 - acc: 0.8784 - val_loss: 0.3875 - val_acc: 0.8276
Epoch 5/15
 - 135s - loss: 0.2342 - acc: 0.9055 - val_loss: 0.4063 - val_acc: 0.8308
Epoch 6/15
 - 132s - loss: 0.1818 - acc: 0.9292 - val_loss: 0.4571 - val_acc: 0.8308
Epoch 7/15
 - 124s - loss: 0.1394 - acc: 0.9476 - val_loss: 0.5458 - val_acc: 0.8177
Epoch 8/15
 - 126s - loss: 0.1062 - acc: 0.9609 - val_loss: 0.5950 - val_acc: 0.8133
Epoch 9/15
 - 133s - loss: 0.0814 - acc: 0.9712 - val_loss: 0.6440 - val_acc: 0.8218
Epoch 10/15
 - 134s - loss: 0.0628 - acc: 0.9783 - val_loss: 0.6525 - val_acc: 0.8138
Epoch 11/15
 - 136s - loss: 0.0514 - acc: 0.9822 - val_loss: 0.7252 - val_acc: 0.8143
Epoch 12/15
 

OK, vamos avaliar a precisão do nosso modelo:

In [9]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=32,
                            verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

Test score: 0.9316869865119457
Test accuracy: 0.80904


81% eh? Nada mal, considerando que nos limitamos às primeiras 80 palavras de cada revisão.

Mas novamente - pare e pense sobre o que acabamos de fazer aqui! Uma rede neural que pode "ler" comentários e deduzir se o autor gostou do filme ou não com base nesse texto. E leva em consideração o contexto de cada palavra e sua posição na revisão - e a configuração do modelo em si foi apenas algumas linhas de código! É incrível o que você pode fazer com o Keras.